In [11]:
import requests
from bs4 import BeautifulSoup
import os

In [12]:
def extraer_todo_a_txt(url="https://misutmeeple.com/2018/08/resena-sagrada/"):
    """
    Esta función extrae contenido estructurado de una reseña de juego de mesa desde el sitio misutmeeple.com.
    Utiliza requests para obtener el HTML, BeautifulSoup para analizarlo, y guarda los datos relevantes en
    un archivo de texto llamado 'resultado_scraping.txt'. También descarga las primeras 5 imágenes del artículo
    en una carpeta local 'imagenes_scrapeadas'.

    Contenido extraído:
    - Título de la pestaña (tag <title>)
    - Encabezado principal (tag <h1>)
    - Subtítulos (h2, h3, h4)
    - Párrafos de texto no vacíos
    - Palabras en negrita (tag <strong>)
    - URLs de imágenes dentro del contenido
    - Comentarios (autor y texto)
    - Ítems de listas (li)
    - Leyendas de imágenes (tag <figcaption>)
    """
    resultados = []  # Lista para acumular el contenido extraído

    # 1. Obtener el HTML de la página
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.text
        resultados.append("✅ Página obtenida con éxito\n")
    else:
        resultados.append(f"❌ Error al acceder a la página: {response.status_code}\n")
        return  # No continuar si hubo error

    # 2. Crear objeto BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # 3. Título y encabezados
    titulo = soup.find("title").text.strip()
    resultados.append(f"Título de la pestaña: {titulo}\n")

    heading = soup.find("h1").text.strip()
    resultados.append(f"H1: {heading}\n")

    # H2, H3, H4
    for nivel, tag in zip(["H2", "H3", "H4"], ["h2", "h3", "h4"]):
        encabezados = soup.find_all(tag)
        resultados.append(f"\n{nivel} encontrados:\n")
        for i in encabezados:
            resultados.append(f"- {i.text.strip()}")

    # 4. Párrafos
    parrafos = soup.find_all("p")
    resultados.append("\n\nPárrafos:\n")
    for i, parrafo in enumerate(parrafos, 1):
        texto = parrafo.text.strip()
        if texto:
            resultados.append(f"{i}. {texto}")

    # 5. Palabras importantes en negrita (<strong>)
    strongs = soup.find_all("strong")
    resultados.append("\n\nPalabras en negrita:\n")
    for i in strongs:
        texto = i.text.strip()
        if texto:
            resultados.append(f"- {texto}")

    # 6. Imágenes
    resultados.append("\n\nURLs de Imágenes encontradas:\n")
    imagenes = soup.find(class_="entry-content-wrap").find_all("img")
    os.makedirs("imagenes_scrapeadas", exist_ok=True)
    for i, img in enumerate(imagenes, 1):
        img_url = img.get("src")
        if img_url.startswith("/"):
            img_url = url + img_url  # Convertir URL relativa a absoluta
        resultados.append(f"Imagen {i}: {img_url}")

        # Descargar las primeras 5 imágenes
        if i <= 5:
            img_data = requests.get(img_url).content
            with open(f"imagenes_scrapeadas/imagen_{i}.jpg", "wb") as f:
                f.write(img_data)

    # 7. Comentarios
    comentarios = soup.select(".comment-body")
    resultados.append("\n\nComentarios:\n")
    for i, comentario in enumerate(comentarios, 1):
        autor = comentario.find(class_="fn").text.strip()
        texto = comentario.find(class_="comment-content").text.strip()
        resultados.append(f"{i}. AUTOR: {autor}\nCOMENTARIO: {texto}\n")

    # 8. Listas (<li>)
    elementos = soup.select(".single-content")
    resultados.append("\n\nElementos de listas:\n")
    for ul in elementos:
        lista_li = ul.find_all("li")
        for li in lista_li:
            resultados.append(f"- {li.text.strip()}")

    # 9. Leyendas de imágenes (<figcaption>)
    leyendas = soup.find_all("figcaption")
    resultados.append("\n\nLeyendas de imágenes:\n")
    for i in leyendas:
        texto = i.text.strip()
        if texto:
            resultados.append(f"- {texto}")

    # 10. Guardar todo en un archivo .txt
    with open("resultado_scraping.txt", "w", encoding="utf-8") as f:
        f.write("\n".join(resultados))

    print("✅ Todo el contenido ha sido extraído y guardado en 'resultado_scraping.txt'.")

# Ejecutar
extraer_todo_a_txt()


✅ Todo el contenido ha sido extraído y guardado en 'resultado_scraping.txt'.
